## Niryo Ned robot control experiments
In this notebook we explore the use of pyniryo robot arm for various behaviour planning, control and management

### Connet robot
connect to the robot based on the IP address of the robot, you can either connect to the robot through the robots hotspot or connect the robot to a shared network between your computer and the robot

In [1]:
import pyniryo
# if you are connected to the robot through the robot's hotspot this is the 
# default ip address for the hotspot connection
# although we need to explore the possibility of connecting the robot to a shared network
# to computer make network requests while connected to the robot
ROBOT_IP_ADDRESS = "172.20.10.3"
# ROBOT_IP_ADDRESS = "10.10.10.10"

# If the computer is not connected to the same network as the robot it raises a network connection exception
try:
    Robot = pyniryo.NiryoRobot(ROBOT_IP_ADDRESS)
except Exception as e:
    print("You are not connected to the same network as the robot")


Connected to server (172.20.10.3) on port: 40001



In [126]:
# Disconnect Robot
Robot.close_connection()


Disconnected from robot



###  Initial robot Setup process
- Calibrate the robot
- Check the learning mode
- Get the hardware status of the robot
- Check the tool in use
- Move the robot to  home position

In [2]:
# Calibrate the robot
Robot.calibrate(calibrate_mode=pyniryo.CalibrateMode.AUTO)

When the learning mode is set to false you are not able to manually adjust the robot joints as they become locked to a particular position

In [84]:
# Experiment with adjusting the learnining mode
Robot.set_learning_mode(True)
current_learning_mode = Robot.get_learning_mode()
print(current_learning_mode)

True


In [5]:
# Check the hardware status of the robot
hardware_status = Robot.get_hardware_status()
# the hardware status shows information such as 
# - Temperature
# - Calibration needed or in progress
# - Motor info, names, temperatures and voltages
# - It also includes potential hardware errors
print(hardware_status)

Temp (°C) : 44.0
Hardware version : 1
Connection Up : True
Error Message : ''
Calibration Needed : False
Calibration in progress : False
MOTORS INFOS : Motor1, Motor2, Motor3, Motor4, Motor5, Motor6,
Names : ['joint_1', 'joint_2', 'joint_3', 'joint_4', 'joint_5', 'joint_6', 'Tool']
Types : ['Niryo Stepper', 'Niryo Stepper', 'Niryo Stepper', 'DXL XL-430', 'DXL XL-430', 'DXL XL-320', 'DXL XL-320']
Temperatures : [30.0, 30.0, 27.0, 33.0, 32.0, 26.0, 28.0]
Voltages : [0.0, 0.0, 0.0, 11.2, 11.2, 7.3, 7.2]
Hardware errors : [0, 0, 0, 0, 0, 0, 0]


In [6]:
# Ensure the tool is updated an get the current tool in use
Robot.update_tool()
tool = Robot.get_current_tool_id()
print(tool) # ToolID.GRIPPER_1 is the tool we are using for our 
            # robot experiment and it is a small gripper

ToolID.GRIPPER_1


In [9]:
# Move the robot to the default location, this is done to ensure movement is
# based on what is defined from the origin
# Set to the learning mode to falseto prevent external adjustment during trajectory
Robot.set_learning_mode(False)
# adjust the pose to move to the home location
Robot.move_to_home_pose()
# set the leanring mode back to true to allow external adjustment
Robot.set_learning_mode(True)

### Manipulating the Tool values and adjusting tool behaviour
In this section we will evaluate tool behaviour and compare and constrast low level tool APIs over  high level grasp and release APIs

In [4]:
# update the current robot tool
Robot.update_tool()
# This ensures the robot is aware of what tool is currently in use

In [5]:
# get the current tool id
current_tool = Robot.get_current_tool_id()
print(current_tool)

ToolID.NONE


In [6]:
# reboot tool
# the tool reboot is used when we want to reset the tool after it gets stuck
Robot.tool_reboot()

In [10]:
# Using high level grasp and release APIs
# GRASP
Robot.grasp_with_tool()
# wait for a few seconds
Robot.wait(3)
# RELEASE
Robot.release_with_tool()
# wait for a few seconds
Robot.wait(3)
# GRASP
Robot.grasp_with_tool()

In [105]:
# Using low level open and close gripper on niryo robot
# The only adjustable parameter we can use for our niryo robot is the speed
gripper_speed = 1000

# the low level gripper API is not working
# GRASP
Robot.close_gripper()
# wait for a few seconds
Robot.wait(3)
# RELEASE
Robot.open_gripper()
# wait for a few seconds
Robot.wait(3)
# GRASP
Robot.close_gripper()

Speed difference across the various speeds is very small and negligible, it would be better to use the high level grasp with tool and release with tool APIs

### Movement, Joints, Poses and Trajectories
In this section we will cover robot movement and pose movement, this will be valuable for us to map the entire scope of location where the robot can move to and map it to pixel images

In [99]:
# check if the robot needs calibration and if it needs calibration perform the calibration
Robot.calibrate_auto()
Robot.wait(2)
Robot.set_learning_mode(False)
Robot.wait(2)
Robot.move_to_home_pose()
Robot.wait(2)
Robot.set_learning_mode(True)

In [18]:
# Joint movement experiments
home_pose = Robot.get_pose()
print(home_pose)

x = 0.1239, y = -0.0001, z = 0.1717
roll = -0.005, pitch = 1.026, yaw = -0.001


In [108]:
# move the joint to a specific joint position
Robot.move_joints(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
Robot.wait(1)
Robot.move_joints(0.2, -0.3, 0.1, 0.0, 0.5, -0.8)
Robot.release_with_tool()
Robot.wait(2)
Robot.grasp_with_tool()
Robot.wait(1)
Robot.move_to_home_pose()
Robot.wait(1)
Robot.set_learning_mode(True)

NiryoRobotException: Command KO : An error occured while executing the last command : Goal has been aborted : Command has been aborted due to a collision or a motor not able to follow the given trajectory

In [54]:
# Set learning mode to true
Robot.set_learning_mode(True)

In [30]:
# Get current pose
# Robot.move_to_home_pose()
Robot.set_learning_mode(False)
pose_read = Robot.get_pose()
print(pose_read)
Robot.set_learning_mode(True)

x = 0.1417, y = 0.0011, z = 0.0900
roll = 0.004, pitch = 1.398, yaw = 0.111


In [ ]:
# Pose object definitions
# x is the vertical distance of the robot end effector to its base, the lower the value of x, the closer the end effector is to the base of the robot 0.14 is the safest posible value for x, the highest save value for x is 0.4, the z values change depending on teh x values
# y is the distance along the horizontal axis of the robot, negetive values are left of the robot
# z determines how high from the base the robot should go, the higher the value of z the higher the robot arm, the lower the value the lower the robot goes, we discovered that 0.09 is the lowest safe value for the robot, we also found that 0.225 id the safest high value that the robot can go to

# The roll, pitch and yaw are best left as they are

# y base 
# x = 0.1417, y = 0.0011, z = 0.0900
# roll = 0.004, pitch = 1.398, yaw = 0.111

# y edge 
# x = 0.4541, y = -0.0222, z = 0.0749
# roll = -0.095, pitch = 0.828, yaw = -0.063


In [35]:
y_base_pose = pyniryo.PoseObject(x=0.14, y=0, z=0.15, roll=0.1, pitch=1.57, yaw=0)
Robot.move_pose(y_base_pose)

In [53]:
y_base_pick_pose = pyniryo.PoseObject(x=0.16, y=0.0, z=0.08, roll=0.1, pitch=1.57, yaw=0)
Robot.pick_from_pose(y_base_pick_pose)

In [54]:
pose_target_obj = pyniryo.PoseObject(
   x = 0.23, y = -0.0000, z = 0.1706,
   roll = -0.010, pitch = 1.032, yaw = -0.001
)
Robot.pick_from_pose(pose_target_obj)

In [124]:
place = pyniryo.PoseObject(
    x = 0.2636, y = 0.0121, z = 0.0630,
    roll = -0.100, pitch = 1.570, yaw = 0.000
)
Robot.move_pose(place)

In [9]:
# moving to a specific pose
# can move to pose as long as poses are valid and can be computed via move-it
Robot.grasp_with_tool()
Robot.move_pose(place)

In [55]:
# Working with joint values to control robot movement
j1 = 0.0 # this joint is responsible for the robot base joint
j2 = 0.0 # this joint is responsible for the robot shoulder joint
j3 = 0.0 # this joint is responsible for the robot elbow joint
j4 = 0.0 # this is responsible for the post shoulder arm twist
j5 = 0.0 # does not control any joints
j6 = 0.0 # controls the wrist joint

Robot.move_joints([j1, j2, j3, j4, j5, j6])


In [55]:
joint_values = Robot.get_joints()
print(joint_values)

[-0.08088698326484066, -0.5448484777320692, -0.9516727014045179, 0.06757914863722043, 0.032253684576855214, -0.11641346110802178]


In [89]:
pose_values = Robot.get_pose()
print(pose_values)

x = 0.2311, y = -0.2440, z = 0.0720
roll = -0.148, pitch = 1.005, yaw = -0.803


In [18]:
x_factor = 0.1140625
y_factor = 0.114583333333333

In [87]:
def calculate_robot_x_axis(pixel_y, pixel_y_base=50, 
                           robot_x_base=0.1413):
    """
    This will take the y axis of the pixel image and calculate the robot x axis
    The pixel y base is the base / minimum value which the robot can reach this should be set at the beginning of the experiment
    THe robot x base is robot x pose value for the provided pixel base
    """
    CONVERSTION_VALUE = 0.0009135
    # check if pixel value is within range
    if (pixel_y < pixel_y_base or pixel_y > 315):
        raise Exception("Pixel Y is out of robot range")
    # get difference from base
    difference = abs(pixel_y - pixel_y_base)
    
    # multiply difference by conversion factor
    robot_axis_difference = difference * CONVERSTION_VALUE
    
    # add robot axis difference to robot axis base
    robot_x_axis = robot_x_base + robot_axis_difference
    return robot_x_axis

In [91]:
calculate_robot_x_axis(300)

0.376175

In [92]:
def calculate_robot_y_axis(pixel_x, 
                           pixel_x_base=110, 
                           robot_y_base=-0.2440):
    """
    This will take the x axis of the pixel image and calculate the robot y axis
    The pixel x base is the base / minimum value which the robot can reach this should be set at the beginning of the experiment
    THe robot y base is robot x pose value for the provided pixel base
    """
    
    CONVERSTION_VALUE = 0.001006968
    # check if pixel is within the desired range
    if (pixel_x < 110 or pixel_x > 640):
        raise Exception("Pixel X is out of robot range")
    
    # calculate difference from base
    pixel_difference = abs(pixel_x - pixel_x_base)
    
    # multiply difference by conversion factor
    robot_axis_difference = pixel_difference * CONVERSTION_VALUE
    
    # add robot axis difference to robot base
    robot_y_axis = robot_y_base + robot_axis_difference
    return robot_y_axis
    

In [93]:
calculate_robot_y_axis(110)

-0.244

In [72]:
def getRobotPoseFromPixelValues(pixel_x, pixel_y):
    y = calculate_robot_y_axis(pixel_x)
    x = calculate_robot_x_axis(pixel_y)
    
    return pyniryo.PoseObject(
        x=x, y=y, z=0.063,
        roll=-0.1, pitch=1.57, yaw=0.0,
        )

In [118]:
pose = getRobotPoseFromPixelValues(313, 187)
print("The calculated pose is ", pose)
Robot.pick_from_pose(pose)

The calculated pose is  x = 0.2729, y = -0.0396, z = 0.0650
roll = -0.100, pitch = 1.570, yaw = 0.000


In [125]:
Robot.move_to_home_pose()
Robot.set_learning_mode(True)